# Importing

Importing takes some time, because Numba just-in-time precompilation is executed. This allows to speed up the code ~5-10 times.

In [1]:
import numpy as np
np.random.seed(42)

from planar_ising import PlanarGraphEdges, PlanarGraph, Triangulator, \
        PlanarGraphGenerator, PlanarGraphConstructor
from planar_ising.lipton_tarjan import PlanarSeparator

# Graph creation

There several ways to obtain a `PlanarGraph` instance for its further separation.

The most low-level way is to use `PlanarGraph` initialization. The following code creates a "triangle" planar graph:

In [2]:
# Create edges list
edges = PlanarGraphEdges(3)

# Add edges between vertices (0, 1), (1, 2) and (2, 0)
edges.append(0, 1)
edges.append(1, 2)
edges.append(2, 0)

# The next line is read as "for edge 0 set edge 1 as the next when rotating
# around vertex 0"
edges.set_next_edge(0, 0, 1)
edges.set_next_edge(0, 1, 2)
edges.set_next_edge(2, 2, 1)

# For each vertex detect one edge incident to it
incident_edge_example_indices = np.array([0, 0, 2], dtype=np.int32)

# Set uniform vertex costs
vertex_costs = np.array([1/3, 1/3, 1/3], dtype=np.float32)

# Create the graph
triangle_graph = PlanarGraph(vertex_costs, incident_edge_example_indices, edges)

A more convenient way to define a graph is through the following helper method. We just pass a list, where for each vertex we enumerate its adjacent vertices in the order of rotation.

In [3]:
triangle_graph = PlanarGraphConstructor.construct_from_ordered_adjacencies(
        [[1, 2], [0, 2], [0, 1]])

Here is the code creating a 2x2 square lattice graph with 9 vertices:

In [4]:
lattice = PlanarGraphConstructor.construct_from_ordered_adjacencies(
        [[1, 3],
         [2, 4, 0],
         [5, 1],
         [0, 4, 6],
         [1, 5, 7, 3],
         [2, 8, 4],
         [3, 7],
         [6, 4, 8],
         [7, 5]])

We can now take a subgraph of it, for example its face on vertices 0-1-4-3:

In [5]:
new_vertices_mapping, new_edge_indices_mapping, square = \
        PlanarGraphConstructor.construct_subgraph(lattice,
        np.array([True, True, False, True, True, False, False, False, False]),
        np.repeat(True, 12))

def output_graph(graph):

    for v in range(graph.size):
        print('{0} is adjacent to {1}'.format(v,
                list(graph.get_adjacent_vertices(v))))

# Note that vertex indices may change
output_graph(square)

0 is adjacent to [1, 2]
1 is adjacent to [3, 0]
2 is adjacent to [0, 3]
3 is adjacent to [1, 2]


Finally, we can generate random planar graphs (see docstrings for the generation algorithm description). Let's generate a graph on 20 vertices with density of 0.8: 

In [6]:
random_graph = PlanarGraphGenerator.generate_random_graph(20, 0.8)

# Planar separation

It's extremely simple to run Lipton-Tarjan algorithm on the planar graph instance:

In [7]:
PlanarSeparator.mark_separation(random_graph)

array([<SeparationClass.SEPARATOR: 2>, <SeparationClass.SEPARATOR: 2>,
       <SeparationClass.FIRST_PART: 0>, <SeparationClass.SEPARATOR: 2>,
       <SeparationClass.FIRST_PART: 0>, <SeparationClass.FIRST_PART: 0>,
       <SeparationClass.SECOND_PART: 1>, <SeparationClass.SECOND_PART: 1>,
       <SeparationClass.FIRST_PART: 0>, <SeparationClass.FIRST_PART: 0>,
       <SeparationClass.SEPARATOR: 2>, <SeparationClass.FIRST_PART: 0>,
       <SeparationClass.SECOND_PART: 1>, <SeparationClass.FIRST_PART: 0>,
       <SeparationClass.FIRST_PART: 0>, <SeparationClass.FIRST_PART: 0>,
       <SeparationClass.FIRST_PART: 0>, <SeparationClass.SEPARATOR: 2>,
       <SeparationClass.SEPARATOR: 2>, <SeparationClass.FIRST_PART: 0>], dtype=object)

# Triangulation

Triangulation is a by-product of Lipton-Tarjan algorithm, which requires it on one of its stages. Let's generate a random tree and triangulate it:

In [8]:
tree = PlanarGraphGenerator.generate_random_tree(5)
output_graph(tree)

0 is adjacent to [1]
1 is adjacent to [0, 4, 2]
2 is adjacent to [1, 3]
3 is adjacent to [2]
4 is adjacent to [1]


In [9]:
new_edge_indices_mapping, triangulated_tree = Triangulator.triangulate(tree)
output_graph(triangulated_tree)

0 is adjacent to [1, 4]
1 is adjacent to [0, 4, 2, 3, 4]
2 is adjacent to [1, 4, 3]
3 is adjacent to [2, 4, 1]
4 is adjacent to [1, 0, 1, 3, 2]
